In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
# use this to keep the cells wider, very nice on wide screens, set the percentage as you like it
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:

conf = {
    'font.size': 14.0,
    'axes.grid': True,
    'axes.axisbelow': True,
    'axes.edgecolor': 'black',
#    'axes.facecolor': '#E5E5EF',
#    'axes.facecolor': '#E0E0EF',
    'axes.labelcolor': 'black',
    'axes.titlesize': 15.6,
    'axes.labelsize': 'large',    
    'figure.figsize': (12, 7),
    'figure.titlesize': 'x-large',
    'grid.linewidth': 1.3,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 'large',
    'xtick.color': 'black',
    'ytick.color': 'black',
    'legend.fontsize': 'large',
    'figure.figsize': (12., 8.)
}

plt.rcParams.update(conf)

In [ ]:
df_form = pd.read_excel('Feedback_Animefest_2019.xlsx', sheet_name='Form Responses 1')

In [ ]:
df_form['Timestamp'] = pd.to_datetime(df_form['Timestamp'])

In [ ]:
df_app = pd.read_excel('Feedback_Animefest_2019.xlsx', sheet_name='Appky')

In [ ]:
df_app['čas feedbacku'] = pd.to_datetime(df_app['čas feedbacku'])

In [ ]:
df_form.head()

In [ ]:
df_form.describe()

In [ ]:
[(i, len(df_form[i].unique())) for i in df_form.columns if len(df_form[i].unique()) < 10]

In [ ]:
df_form.columns.map(lambda i: len(df_form[i].unique())).to_series().value_counts().sort_index().plot.bar(figsize=(25, 10))

In [ ]:
df_form.columns.map(lambda i: sum(df_form[i].notnull())).to_series().value_counts().sort_index().plot.bar(figsize=(25, 10))

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) <= 8])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) < 10])

In [ ]:
len([i for i in df_form.columns if len(df_form[i].unique()) < 20])

In [ ]:
df_app.head()

In [ ]:
df_app.describe()

In [ ]:
def abs_vals(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return '{:d}'.format(absolute)


In [ ]:
feedback_geo = df_form['Bydliště'].value_counts()
feedback_geo.plot.pie(autopct=partial(abs_vals, allvals=feedback_types))

In [ ]:
for i in [i for i in df_form.columns if len(df_form[i].unique()) <= 8]:
    plt.figure()
    feedback_i = df_form[i].value_counts()
    feedback_i.plot.pie(autopct=partial(abs_vals, allvals=feedback_i))
    plt.show()

In [ ]:
feedback_ages = df_form['Věk'].value_counts()
feedback_ages.plot.pie(autopct=partial(abs_vals, allvals=feedback_types))

In [ ]:
feedback_genders = df_form['Pohlaví'].value_counts()
feedback_genders.plot.pie(autopct=partial(abs_vals, allvals=feedback_types))

In [ ]:
feedback_app = df_form['Používali jste Animefest aplikaci?'].value_counts()
feedback_app.plot.pie(autopct=partial(abs_vals, allvals=feedback_types))

In [ ]:
feedback_app = df_form['Cena AF suvenýrů je přiměřená?'].value_counts()
feedback_app.plot.pie(autopct=partial(abs_vals, allvals=feedback_types))

In [ ]:
feedback_app = df_form['Cena vstupného je'].value_counts()
feedback_app.plot.pie(autopct=partial(abs_vals, allvals=feedback_types))

In [ ]:
#feedback_times = df_form['Timestamp'].value_counts()
#feedback_times.plot.pie(autopct=partial(abs_vals, allvals=feedback_types))
feedback_times = df_form['Timestamp'].value_counts(sort=False).copy().reindex().resample('1D', how='count')
feedback_times.plot.bar()
plt.title('Počet vyplnění za den, vše červen')
#plt.gca().set_xticklabels([dt.strftime("%#d. %#m.") for dt in feedback_times.index])
plt.gca().set_xticklabels([dt.strftime("%#d.") for dt in feedback_times.index])
plt.xticks(rotation=0)
plt.show()

In [ ]:
#feedback_times = df_form['Timestamp'].value_counts()
#feedback_times.plot.pie(autopct=partial(abs_vals, allvals=feedback_types))
feedback_times = df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'].value_counts(sort=False).copy().reindex().resample('1D', how='count')
feedback_times.plot.bar()
plt.title('Počet vyplnění za den, vše červen')
#plt.gca().set_xticklabels([dt.strftime("%#d. %#m.") for dt in feedback_times.index])
plt.gca().set_xticklabels([dt.strftime("%#d.") for dt in feedback_times.index])
plt.xticks(rotation=0)
plt.show()

In [ ]:
from urllib import request

data = request.urlopen('https://raw.githubusercontent.com/stopwords-iso/stopwords-cs/master/stopwords-cs.txt')
all_stopwords = [line.decode('utf-8').replace('\n', '') for line in data]
# todo: split them correctly and for vocalulary calculation use only emotes filtering
emotes_stopwords = [':d', ':', '\?', ';', ':-d', '\^\^', ':dd', 'xd', ':3', '=d', ':p',  ':/', ':>', '\+', 'o_o', 'd:', '>', ':\'d', ':"d', '\n', ':dd', ':ddd', ':dddd', 'x', 'xp']
my_stopwords = ['no', '', 'jo', 'sa', 'the', '=d', 'fakt', 'celkem', 'sem', 'ja', 'som', 'ako', 'of', 'nekdo', 'bych', 'sou', '2', 'jj', 'is', 'mam', '3', '\n']
all_stopwords += my_stopwords
all_stopwords += emotes_stopwords

def messages_to_words_hist(mess_series):
    mess_series = mess_series.fillna('')
    all_words = mess_series.str.split(' ', expand=True).stack().reset_index(drop=True, level=1)
    words_historgram = all_words.str.lower().str.strip('.,?-–()').value_counts()
    stopwords = list(set(all_stopwords).intersection(set(words_historgram.index)))
    words_historgram_cleaned = words_historgram.drop(stopwords)
    return words_historgram_cleaned


In [ ]:
best_mess = messages_to_words_hist(df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'].str.contains('přednášky', regex=False)]['S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Co vás letos nejvíc zklamalo / nelíbilo se vám?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='Co vás letos nejvíc zklamalo / nelíbilo se vám?')
plt.xticks(rotation=30)

In [ ]:
with pd.option_context('display.max_colwidth', 150):
    display(df_form[df_form['Co vás letos nejvíc zklamalo / nelíbilo se vám?'].str.contains('přednášky', regex=False)]['Co vás letos nejvíc zklamalo / nelíbilo se vám?'])

In [ ]:
best_mess = messages_to_words_hist(df_form['Co bychom podle vás měli do příštího roku zlepšit?'])
best_mess[:30].plot.bar(figsize=(30, 10), title='Co bychom podle vás měli do příštího roku zlepšit?')
plt.xticks(rotation=30)